<a href="https://colab.research.google.com/github/delibae/health_care_pr/blob/minsoo/Data_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import cv2
import numpy as np
import glob
import pickle
import json
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

In [7]:
input_path = '/content/drive/MyDrive/data/두피/Image_hairloss/*.jpg'
file_list = [f for f in glob.glob(input_path)] 

In [ ]:
def make_path(path) :

    result = []

    for i in path :
        
        for j in glob.glob(i) :
            result.append(j)

    return result

In [15]:
def handle_image(path) :

    reshaped_image = []

    for i in tqdm(path) :
        
        image_bgr = cv2.imread(i, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        image_50x50 = cv2.resize(image_rgb, (50, 50))

        reshaped_image.append(image_50x50)

    return reshaped_image

image = handle_image(file_list)
iamge = np.array(image)

100%|██████████| 3813/3813 [17:15<00:00,  3.68it/s]


In [ ]:
input_path = '/content/drive/MyDrive/data/두피/Label_hairloss/*.json'
file_list1 = [f for f in glob.glob(input_path)]

file_list1

In [35]:
def handle_json(path) :

    result = []

    for p in tqdm(path) :
        with open(p, 'r',encoding="UTF-8") as file:
            data = json.load(file)
            data_path = data['image_file_name']
            value = []
            for i in range(1, 7) :
                k = f'value_{i}'
                value.append(int(data[k]))
          
            result.append(value)

    return result
        
label = handle_json(file_list1)
label = result2 = np.array(label)

100%|██████████| 3813/3813 [16:38<00:00,  3.82it/s]


In [51]:
def softmax(data) :
    exp_data = np.exp(data)

    return exp_data / np.sum(exp_data, axis=1, keepdims=True)

softmax_data = softmax(result2)

assert np.isclose(np.sum(softmax_data, axis=1), np.ones(result2.shape[0])).all()

result2 = softmax_data

In [57]:
result2.shape

(3813, 6)

In [61]:
import torch
from torch.utils.data import TensorDataset, DataLoader

image = torch.tensor(image)
label = torch.tensor(label)

In [ ]:
dataset = TensorDataset(image, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)